___________________03_Hunt_Freyberg_Beta .xlsx
___stoped at _______________________________________CTRL F Rules for constructing INS Files

In [34]:
import sys
import os, shutil
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt;
# sys.path.insert(0,os.path.join("..", "..", "dependencies"))
import pyemu
import flopy
#assert "dependencies" in flopy.__file__
#assert "dependencies" in pyemu.__file__
sys.path.insert(0,"..")
import herebedragons as hbd

In [35]:
org_d = os.path.join('..', '..', 'models', 'monthly_model_files_1lyr_newstress') # defining ____org
tmp_d = os.path.join('pest_files') # ____________________________________________# geting the same folder ... pest_files = ``freyberg_mf6``

if os.path.exists(tmp_d): # ____________________________tmp_d (?)
    shutil.rmtree(tmp_d)
shutil.copytree(org_d,tmp_d) #____ conection ____org  x tmp 

hbd.prep_bins(tmp_d) #______ get executables _______ MODFLOW6  MP7   INSCHECK ___________EX E:\03_pest_docs\05_CODES\04_CODE_PEST\pestchek.F_2009
hbd.prep_deps(tmp_d) #______get dependency folders
hbd.prep_pest(tmp_d) #______ run ____
# ____________ heads.csv.ins heads.csv  _______ sfr.csv.ins sfr.csv  _________________`` template & (input) ``

mf6.exe
mp7.exe freyberg_mp.mpsim
inschek.exe heads.csv.ins heads.csv
ins file for heads.csv prepared.
inschek.exe sfr.csv.ins sfr.csv
ins file for sfr.csv prepared.
noptmax:0, npar_adj:1, nnz_obs:24
written pest control file: pest_files\freyberg.pst


___________________________________________________________________________________

In [ ]:
hbd.plot_freyberg(tmp_d)

In [ ]:
!head -n 15 'pest_files\freyberg.pst'

``RIV = SFR`` | GHB | No-flow | Pumping | Time-series (x) | ``RIV flux 03 locations`` - ``headwater`` - ``tailwater`` - ``gage`` ... not displayed
``previous tutorial trial-and-error``
Template files (*.tpl & *.ins) | parameters | control file (points out all others templates), ...    `freyberg.pst` 

"`ptf ~`" ___________PTF____________________________________________Parameters "`~    hk1    ~` " will be replaced by the value for `hk1` 

>>```
>>* model input/output
>>./freyberg_mp.ne_layer1.txt.tpl ./freyberg_mp.ne_layer1.txt
>>./freyberg6.npf_k_layer1.txt.tpl ./freyberg6.npf_k_layer1.txt
>>./freyberg6.rch.tpl ./freyberg6.rch
>>./sfr.csv.ins ./sfr.csv
>>./heads.csv.ins ./heads.csv
>>./freyberg_mp.mpend.ins ./freyberg_mp.mpendfreyberg6.npf_k_layer1.tpl 
>>```

In [29]:
# pstB = pyemu.Pst(os.path.join("pest_files/freyberg.pst"))    # pstB = pyemu.Pst(os.path.join(tempNEW,"freyberg.pst"))  # Intro to pyEMU
# pstB.parameter_data.head() # pstB.observation_data.head() # pstB.control_data.formatted_values # Intro to pyEMU

TEMPcheck ___________________________________________________________________________________templates files

In [ ]:
prompt at ..\pest_files PESTcheck freyberg6.rch.tpl ____*.pmt _______________________________parameter files
prompt at ..\pest_files TEMPcheck freyberg6.npf_k_layer1.txt.tpl   .npf_k_layer1.tpl  .freyberg6.rch.tpl                 #### Visual Studio
E:\15_REPOS\00_BETAMI\01_GMDSI_____advanced\01_Tutorials___AGAIN___\part1_02_pest_setup_NOTYET__W_inprogress\pest_files  # Path

INSCHEK ___________________________________________________________________________________instruction files

>```
>heads.csv.ins heads.csv
>sfr.csv.ins sfr.csv
>```

### Rules for constructing INS Files 

 * The first line on an .ins file must be "`pif ~`" where "`~`" is a "marker delimiter"--a symbol that can be used to identify text to search for.  It is expected on this first line but it's not always used.
 * The scripting options are extensive but particular. Some options on how to navigate to the numerical data you want to read are:
   1. Using a line advance.  PEST starts at the very top the model output file, and you have to move down 1 line to get to the first line that you see in the model output. You can tell PEST to move down the file _`n`_ lines using the `l` character (=lowercase letter l) with a number.  So "`l1`" moves down one line, "`l3`" moves down 3 lines.  
   2. Using the marker delimiter, the INS file can search through a file until it finds a "primary marker". For example:  
   "`~VOLUMETRIC BUDGET FOR ENTIRE MODEL~`" can be used to search for budgets in a LST file  
   This is particularly well suited for output files (like a LST file) that have unpredictable lengths.  Note though that PEST will always start at the top of the file and go down, never up and never wrapping once it reaches the end.  This can be a problem when the order of some observations with respect to other observations is not consistent (e.g., some MODPATH output).  When searching for mutiple observations that may vary in order in an output file, it is easiest to have multiple instruction files open the same model output file multiple times so you are always starting at the top of the file (PEST does not mind). 
   3. Next, you can search for a "secondary marker" within a line using the marker delimiter again. This navigates from the left of the line until the secondary marker is found.
   4. Once on the line you can specify which columns on a line to extract.  So a line in an instruction file that says `~101  138~ (depth_T2-90)46:58` means that PEST will look for `101  138` in the model output file (with the exact number of spaces between the two numbers) then extract column `46:58` to determine the model output that equates to the target observation `depth_T2-90` that is listed in the PEST control file.   
5. Finally, you can read in whitespace-delimited numerical data using "`!`" around the observation name:  
   for example, if the output file is:  
      > ```
      > Output file from run 5
      > Run date: 1/1/2012
      > Run time: 24.3 hours
      > Converged in 350 iterations
      > Head Observations:
      > H1=33.345 H2=45.34
      > ...
      > ```  
      The instruction file would be like 
      >  ```
      > pif ~
      >  ~Head Observations~
      >  l1 ~H1=~ !h1val! ~H2=~ !h2val!
      > ```
  
These are only a few of the most commonly used options but more options, and more detail on these, are available in the PEST manual.  

**PRO TIP**: _The Instruction File syntax enables the reading of very complicated files which affords great flexibility. However, it is often easier to debug scripts or custom codes to read model output. So, for users familiar with scripting and programming tools, it can be effective to write a postprocessing code to extract observation values and write them out into a simple tabular format for which an instruction file is easy to write and debug._

### 3.2. Checking a template file with the `INSCHEK` utility

Let's check an instruction file using `INSCHEK`, a handy utility that allows us to check our instruction files without having to do a full PEST run. You can see what INSCHEK is looking for by simply typing "inschek" (note lower case) at the command line.  You'll see: 

> ```
> INSCHEK Version 14.01. Watermark Numerical Computing.
> 
> INSCHEK is run using the command:
> 
>     INSCHEK insfile [modfile]
> 
> where
> 
>     "insfile" is a PEST instruction file, and
>     "modfile" is an [optional] model output file to be read by INSCHEK.
> ```

When successful, `INSCHEK` writes an output file with the extension `.obs`, listing observation names that it found in the instruction file. If a model output file name was also specified, `INSCHEK` also writes the observation values it found in that file. This enables a quick check to verify that files have been setup correctly. 

Check the instruction files listed in `freyberg.pst` by running INSCHEK: 
1) without the optional model output file/ look at output; and 
2) with the optional model output file/look at output.  

(Note:  yes the author of PEST John Doherty knows how to spell! He could have made it INSCHECK but chose to be consistent across all his checking programs and for some, like TEMPCHEK above, proper spelling would not fit in the 8.3 filename format required at the time.  The good news is you only have one to remember - just think CHEK.)

### 3.3. Make your own INS file

Test your understanding. Try building your own instruction file. As for TPL files, it is sometimes easiest to start by using a model output as a template.

Remember to always check the validity of the INS file using `INSCHEK`.

### 4. The Control File

So, the previous sections introduced the basics of how to construct TPL and INS files. This will come in handy when you need to build your own customized PEST input datasets.

The last piece is the control file, which you have already seen (the file named `freyberg.pst`). We will not build this file from scratch here. In practice you rarely will. Most GUIs construct a template for you. `pyEMU` does the same. Both the PEST and PEST++ user manuals provide detailed descriptions of the control file and its various sections. We recommend reading them.

Very briefly: 

The control file is divided into sections. Each section of the PEST control file must begin with a single line of text comprising the header to that section. This line must begin with a “*” character followed by a space.  The most common sections which users are likely to access are:

> `* control data` - problem dimensions, mode of PEST operation, termination criteria, change limits and other control data

> `* parameter group` - variables which govern the way in which finite-difference derivatives are calculated

>  `* parameter data` - parameter initial values, transformation status, bounds,
groups, scales and offsets

> `* observation groups` - lists observation groups, and provides names of files holding observation covariance matrices

> `* observation data` - provides measured values of observations, weights and groups to which observations belong

> `* model command line` - provides one or a number of commands used by PEST to run the model in the command line

> `* model input/output` - as we have seen above, lists template and corresponding model input files as well as instruction and corresponding model output files


Take a moment to explore the `freyberg.pst` control file. You will find all the parameters listed in the various TPL files referenced in the `* parameter data` section. You should also find all the observations listed in the INS files listed in the `* observation data` section.

The `* model command line` section provides the command that PEST uses to run the model. Usually this would be a command to run a batch file or script. Here we have simply used the command to run MODFLOW6. You can see for yourself if it works by opening a command line in the "pest_files" folder, typing `mf6` and pressing `<enter>`. (This should run the model once.) 

### Check the control file with PESTCHEK

Just like TEMPCHEK and INSCHEK, we also have a handy utility that we run on our PEST setup before pulling the trigger. 

> **note**: always check your PEST setup with PESTCHEK! Certainly do so before complaining to the developers that something isn't working properly. 

Just like TEMPCHEK and INSCHEK, you can see what PESTCHEK is looking for by simply typing `pestchek` (Windows) or `./pestchek` (Mac) at the command line.  If you did that you would see that we have to put this on the command line to check our PEST setup: __`pestchek freyberg_un.pst`__ (if Windows) or __./pestchek freyberg_un.pst__ (if Mac)

If errors are indicated, PEST won't run so we have to correct them. Warnings, on the other hand, highlight potentially good information about what you have specified in the control file but they don't require a change to run. However, the warnings may guide your eyes to things you are not intending so always read them too.

You should see something like:

>```
>Warnings ----->
>NUMLAM is supplied as negative. This will be reset to positive. If Parallel
>  PEST or BEOPEST is used PARLAM will automatically be set to -9999.
>NOPTMAX provided as zero. No optimisation iterations will be carried out;
>  objective function and residuals will be recorded for initial parameter
>  estimates only.
>All parameters belonging to parameter group "rch" are either fixed or tied.
>MAXSING in the singular value decomposition section is greater than the
>  number of adjustable parameters.
>```

If no errors are highlighted, let's go ahead and run PEST!

### Run PEST from Command Line

From the command line run __`pestpp-glm freyberg.pst`__ (Windows) or __`./pestpp-glm freyberg.pst`__ (Mac).

PEST++ should commence, run the model once and then stop (should take a ouple of seconds at most). You should see something like the following in your terminal:

>```
>-----------------------------------------
>     ---   OPTIMIZATION COMPLETE   ---
>  Reason for terminating PEST++ simulation: NOPTMAX criterion met
>  Summary of termination criteria:
>  NOPTMAX = 0 ;  NOPT at termination = 0
>  NPHINORED = 3 ;  NPHINORED at termination = 0
>  NRELPAR = 3; RELPARSTP = 0.01 ;  NRELPAR at termination = 0
>  PHIREDSTP = 0.01; NPHISTP = 3
>  NPHISTP lowest PHI's:
>
>FINAL OPTIMISATION RESULTS
>
>  Final phi                                           Total : 16916.9
>  Contribution to phi from observation group         "flux" : 0
>  Contribution to phi from observation group          "hds" : 16916.9
>  ```

If you check the "pest_files" folder, you should see several new files. Open the one named "freyberg.rec". This is the PEST record file. It records lots of useful information about the PEST run. 

If you scroll down to the end of the file you should see a line that says:

> `Number of forward model runs performed during optimization: 1`

As mentioned in the PESTCHEK warning, the control file we gave you has `NOPTMAX=0`, which means the model only is run once, and then PEST++ processes all the output and reports the objective function phi. So, not too exciting with only one run.  However, we __always__ run with `NOPTMAX=0` first to "test the plumbing" of the template and instruction files, and to see if we like the contribution of observation groups to the total objective function. If we don't like the objective function distribution we can reweight, then re-run PEST++ with `NOPTMAX=0` again. (We will demonstrate this in a subsequent tutorial.)

###  4.3. Finally - let's get a best fit for this problem!

Now __change `NOPTMAX` to a value = 20__ (`NOPTMAX` is the first number listed in the 9th line of the PEST control file).  You can see its location below, taken from Appendix 1 from SIR 2010-5169 we will be handing out:

<img src="freyberg_pest_setup_files/2010-5169_Appendix1_PST_file.png" style="float: center">

The full listing of the PEST control file and a description of each variable is in Appendix 1. __*However, most of these you never will need to touch - the defaults are fine!*__  NOPTMAX, however, is one that you will routinely touch. 

Now run PESTCHEK again - note that the NOPTMAX=0 warning is now gone.  

**If no errors, run PEST++ again.**

This will run parameter estimation on the model and will max out at 20 parameter upgrades. (This time it should take a couple of minutes.) You may have figured out by now, but NOPTMAX stands for __N__umber of __OPT__imization iterations __MAX__imum --cryptic variable names were the price one had to pay when computer RAM was exceedingly small! 

Check the `freyberg.rec` file again. What has changed? Try searching for "Final phi". How does it compare to the initial Phi (which we obtained earlier by running the modell only once)? It should be a lot smaller. 

>```
>Final composite objective function 
>  Final phi                                           Total : 51.7655
>  Contribution to phi from observation group         "flux" : 0.000000
>  Contribution to phi from observation group          "hds" : 51.7655
>
>Number of forward model runs performed during optimization: 117
>```

What about the best-fit parameters? There should be a file named `freyberg.par`. Open it in a text editor. These are the best parameter values which PEST has managed obtain. Note that all parameters in that were in the control data `* parameter data` section are listed here. However, only the "hk" parameter values have changed because all the others were specified as "fixed". How do these values compare to what you achieved through manual trial-and-error calibration? 

# Moving into the 21st Century
## Thinking about forecast uncertainty 

Recall that in your freyberg_trial_and_error tutorial we were looking at the fit between the model and historical data, but also at a "forecast" of river flow. This forecast was in fact the observation named "headwater:4383.5" (the simulated headwater during the last model stress-period). 

If there is __*one thing*__ we want you to take away from this class it is this:  

**For most models there is a forecast/prediction that someone needs. Rather than waiting until the end of the project, the forecast should be entered into your thinking and workflow __right at the beginning__.**

Open up `freyberg.pst` in a text editor and look in the observation section. Find entry for the "headwater:4383.5" observation. Note that the observation **weight** is zero. So, although we have included this "observation" in the dataset, it does not in fact contribute to the objective function (it does not affect calibration). This is a nifty trick - it means we can put in forecasts __and__ hypothetical observations even if we don't have measured values for them! More on this coming up so stay tuned!   

But wait a minute - how does PEST know we have a forecast and not a calibration data point? PEST++ made this very easy - simply add the forecasts as observations and list them in a ++forecasts section in the PEST control `.pst` file. This was already done in the control file in this exercise.  Find the `++forecasts` command. The `++` at the beginning means this input will ___only be seen by PEST++___; if you use PEST it will be ignored. 


Let's open `freyberg.rec` in a text editor and look at the forecast uncertainty output reported at end of the .rec file.  Look also at the uncertainty reported for the "hk" conductivity parameters. (How exactly was that uncertainty calculated?  Ah, we'll get to that a little later)

To recap:  you just added a forecast and got an estimate of the uncertainty by simply adding one line to the PEST++ control file!

__Note:__ other PEST++ specific input options can be found at https://github.com/usgs/pestpp/blob/master/documentation/pestpp_users_manual.md.  During this course we will touch on the most commonly used ones. 